In [1]:
%%capture
!pip install datasets
!pip install transformers
!pip install faiss-gpu
!pip install nlp
!pip install spacy
!pip --no-cache-dir install nlpaug

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os

# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "./Colab Notebooks/Deep Daiv '23w/"
GOOGLE_DRIVE_PATH = 'deeplearning/deepdai/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH)
print(os.listdir(GOOGLE_DRIVE_PATH))

import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

with open(GOOGLE_DRIVE_PATH+'harry_data2.json') as json_file:
    json_data = json.load(json_file)

Mounted at /content/drive
['DPR_and_TFIDF.ipynb', 'GPTNeo Fine-tuning.ipynb', 'PPO.ipynb', 'harry1.txt', 'harry2.txt', 'harry3.txt', 'harry_info.json', 'Selfcode.ipynb', 'harry_info_one.json', 'crawling.ipynb', 'alpaca_data.json', 'deepdaiharry_sum.txt', 'deepdaiharry_sum.json', 'harry_data.json', 'harry_data2.json', 'Retrieval.ipynb', 'Reward', 'reward_logs2.txt', 'reward_logs3.txt', 'InstructMyself.ipynb', 'Last_serving.ipynb', 'SFT.ipynb', 'PPO_training.ipynb']


### Pre-processing

In [3]:
contractions_dict = { "ain't": "are not","aren't": "are not","can't": "can not","can't've": "cannot have",
    "'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
    "didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
    "hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
    "he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
    "i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
    "isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
    "mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
    "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
    "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
    "she'll've": "she will have","should've": "should have","shouldn't": "should not",
    "shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
    "there'd": "there would","there'd've": "there would have",
    "they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
    "they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
    "what've": "what have","when've": "when have","where'd": "where did",
    "where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
    "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
    "you're": "you are","you've": "you have"}

In [4]:
import re
import spacy
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

def preprocessing_sents(data):
  # replace 'J. K. Rowling'
  # def ContextualWordEmbeddingRoberta(text):
  #   #aug = naw.synonym.SynonymAug(
  #       #aug_src='wordnet', model_path=None, aug_max = 1)
  #   aug = naw.ContextualWordEmbsAug(
  #       model_path='roberta-base', action="insert", aug_max = 1)
    
  #   #aug = naw.ContextualWordEmbsAug(
  #     #model_path='bert-base-cased', action="insert", aug_max = 1)
  #   return aug.augment(text)

  # for sent in data:
  #   if 'J. K. Rowling' in sent:
  #     sent.replace('J. K. Rowling', 'author')
  #   sent = sent.lower()
  #   if 'i' in sent:
  #     sent.replace('i', 'Harry')
  #   if 'my' in sent:
  #     sent.replace('my', "Harry")
  #   if 'me' in sent:
  #     sent.replace('me', "Harry")
  #   sent = ContextualWordEmbeddingRoberta(sent)

  #   # Regular expression for finding contractions
  contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

  def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
      return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
   
  return list(map(lambda x: expand_contractions(x), data))


In [5]:
def preprocessing_query(data):
  # data = data.lower()
  # aug = naw.ContextualWordEmbsAug(
  #       model_path='roberta-base', action="insert", aug_max = 1)
  # # aug = naw.synonym.SynonymAug(
  #       #aug_src='wordnet', model_path=None, aug_max = 1)
  # data = ' '.join(aug.augment(data))
  
  # contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

  #   # Function for expanding contractions
  # def expand_contractions(text,contractions_dict=contractions_dict):
  #   def replace(match):
  #     return contractions_dict[match.group(0)]
  #   return contractions_re.sub(replace, text)
  

  # data =  expand_contractions(data)

  # if 'Harry Potter' in data:
  #    data.replace('Harry Potter', 'I') 
  # if 'Harry' in data:
  #    data.replace('Harry', 'I') 

  if 'you' in data:
    data.replace('you', 'I')
  if 'your' in data:
    data.replace('your', 'My')  
  if "yours" in data:
    data.replace('yours', 'My')
  if "friend" in data:
    data += " friends"
  if "sport" in data:
    data += " sports"
  
  data.replace('are', 'am')
      
  return data

In [6]:
json_data = preprocessing_sents(json_data)

In [7]:
def change_you_to_Harry(query):
  if 'your' in query:
    query=query.replace('your', "My")
  if 'you' in query:
    query=query.replace('you', 'I')
  return query

### Retrieve

In [8]:
HarryDict = dict()
HarryDict['id']=[x for x in range(len(json_data))]
HarryDict['sent']=[x for x in json_data]

from datasets import Dataset
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch

dataset = Dataset.from_dict(HarryDict)

torch.set_grad_enabled(False) # https://runebook.dev/ko/docs/pytorch/generated/torch.set_grad_enabled
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [9]:
# def tokenize_function1(text):
#     return tokenizer(text["1sent"],padding=True, truncation=True, return_tensors='pt')
# def tokenize_function2(text):
#     return tokenizer(text["2sents"],padding=True, truncation=True, return_tensors='pt')
import faiss

embeddings1 = dataset.map(lambda x: {'embeddings': model(**tokenizer(x["sent"],padding=True, truncation=True, return_tensors='pt'))[0][0].numpy()})

embeddings1.add_faiss_index(column='embeddings')

Map:   0%|          | 0/589 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'sent', 'embeddings'],
    num_rows: 589
})

In [10]:
import pandas as pd
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

In [11]:
num_of_sentences = 30

In [12]:
def get_tf_results(queries):
  queries = [change_you_to_Harry(x) for x in queries]
  tf_weight_list = [x for x in range(50, 0, -1)]
  data = queries + json_data

  tfidf_vect = TfidfVectorizer(stop_words='english')
  tfidf_matrix = tfidf_vect.fit_transform(data)
  cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

  tf_result = []
  for idx in range(len(queries)):
    injected_list=[]
    sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:num_of_sentences]
    i=0
    for (index, score) in sim_scores:
      if sim_scores[i][1]==0:
        cal_score=0
      else:
        cal_score=(sim_scores[i][1]/3+1.2)*tf_weight_list[i]
      injected_list.append({"index":index-len(queries), "score":cal_score, "sent":data[int(sim_scores[i][0])]})
      i+=1
    # for i in range(num_of_sentences):
    #     print(f'score : {injected_list[i]["score"]}, Found : {injected_list[i]["sent"]}')
    tf_result.append(injected_list)
  return tf_result

In [13]:
def get_sim_results(queries):
  sim_results=[]
  k=num_of_sentences

  for orig_query in queries:
    query = change_you_to_Harry(orig_query)
    question_embedding = q_encoder(**q_tokenizer(query, return_tensors="pt"))[0][0].numpy()
    scores, retrieved_examples = embeddings1.get_nearest_examples('embeddings', question_embedding, k=k)
    df_dpr = pd.DataFrame.from_dict(retrieved_examples)
    df_dpr["scores"] = scores
    df_dpr.sort_values("scores", ascending=False, inplace=True)
    sim_results.append({
        "query":orig_query,
        "result":df_dpr[['id', 'sent', 'scores']]
    })
  return sim_results

In [14]:
queries = ["Who is your best friend?", "What is your favorite sport?", "Do you like Voldemort?", "Can you tell about Hermione Granger?"]

In [21]:
queries = ["Who is your best friend?", "Do you know about dobby?", "What do", "Can you tell me about your parents?", "What spell do you like most?", "Can you tell about Hermione Granger?"]

In [22]:
import collections
import math

def get_all_result(queries):
  sim_results = get_sim_results(queries)
  queries = [preprocessing_query(x) for x in queries]
  tf_results=get_tf_results(queries)

  all_results=[]
  hyper = 0.25

  for (idx, sim) in enumerate(sim_results):
    print("=-------------------"*4)
    result_dict=collections.defaultdict(int)
    for i in range(len(sim['result'])):
      result_dict[sim['result'].iloc[i]['id']]=[sim['result'].iloc[i]['scores']*(1-hyper), sim['result'].iloc[i]['sent']]
    for tf in tf_results[idx]:
      if result_dict[tf['index']]:
        print("겹침", math.floor(result_dict[tf['index']][0]*100)/100, math.floor(tf['score']*100)/100, tf['sent'])
        result_dict[tf['index']] = [tf['score']*hyper + result_dict[tf['index']][0], result_dict[tf['index']][1]]
      else:
        result_dict[tf['index']]=[tf['score']*(hyper), tf['sent']]
    sorted_dict = dict(sorted(result_dict.items(), key=lambda item: item[1],reverse=True))
    all_results.append(sorted_dict)

  for (i, q) in enumerate(queries):
    print("질문은: ", q)
    res=list(all_results[i].values())[0]
    print("Top 1:", res[1], res[0])
    res=list(all_results[i].values())[1]
    print("Top 2:", res[1], res[0])
    print("\n\n")

  return all_results

In [23]:
all_result=get_all_result(queries)

=-------------------=-------------------=-------------------=-------------------
겹침 71.96 66.4 I was sorted into Gryffindor House at Hogwarts, where I became best friends with Ron and Hermione.
겹침 61.22 63.62 My friends mean everything to me, and I'll always put their needs and best interests first. That's why I ended my relationship with Ginny Weasley, even though being with her was one of the best things that ever happened to me.
=-------------------=-------------------=-------------------=-------------------
겹침 72.51 65.55 I tricked Malfoy into freeing Dobby by giving him one of my socks, which led to Dobby thanking me and leaving.
겹침 72.09 61.17 Dobby was trying to protect me from the monster inside the Chamber of Secrets, which had been opened before.
겹침 72.63 56.77 With the help of Dobby the house-elf, we were able to escape from Malfoy Manor and continue our mission to destroy the Horcruxes.
겹침 66.66 52.83 With the help of Dobby's hints, I exposed Malfoy's plan to frame Ginny fo

In [24]:
all_result

[{34: [88.56735209337461,
   'I was sorted into Gryffindor House at Hogwarts, where I became best friends with Ron and Hermione.'],
  560: [77.13529083361091,
   "My friends mean everything to me, and I'll always put their needs and best interests first. That's why I ended my relationship with Ginny Weasley, even though being with her was one of the best things that ever happened to me."],
  252: [71.97711181640625,
   'I, along with Ron and Hermione, had to rescue Buckbeak from execution and confront Sirius Black, an illegal Animagus.'],
  53: [71.81181335449219,
   'Surviving the Killing Curse made me the "Boy Who Lived" and gave me a lightning-bolt scar on my forehead. It also opened a telepathic link between Voldemort and myself, which would prove to be both a bane and a blessing in the years to come.'],
  14: [71.79461860656738,
   'My greatest weakness is my tendency to be reckless and rush into dangerous situations without thinking.'],
  50: [71.47841262817383,
   'My parents, L